In [1]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

#Imports principais para manipulação e visualização
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import warnings
import plotly.express as px

Mounted at /content/drive


# Parte 1

O time de Experiência precisa que você faça uma análise exploratória completa com os dados a sua disposição, entenda os gargalos, crie os acompanhamentos e indicadores, gere insights, formule hipóteses e proponha um plano de ação.

Entregáveis esperados: análise quantitativa + data storytelling + códigos (sql e/ou python)

In [2]:
#import tabela 1 - dados de atendimento
df1 = pd.read_csv('/content/drive/My Drive/case_will/issues_atendimento_.csv')
df1.head()

,dt_data,id_cliente,id_issue,canal,tag
0,2024-10-07 17:32:50.000,22df4146-1b0b-42e1-bc64-1905e921a56a-765gh,8095572,chat,w3 - reclamacao - resultado analise de contest...
1,2024-12-03 19:09:43.000,019226ac-f859-7590-9cfe-56d56f5e8f59-765gh,9872055,chat,w3 - reclamacao - resultado analise de contest...
2,2024-10-16 18:26:47.000,c73c1daa-efd4-4a07-9671-1e52fb008601-765gh,8428710,chat,w3 - duvida - pix no credito
3,2024-12-12 15:34:49.000,01923e84-7191-7bf0-bd29-0aeb1417f4e1-765gh,10153733,chat,w3 - problema - desacordo comercial
4,2024-12-05 12:37:52.000,881b24fc-23d1-4dd3-80ab-00f636cecdfa-765gh,9934026,chat,wb-caf


In [3]:
#infos tabela 1
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804302 entries, 0 to 804301
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   dt_data     804302 non-null  object
 1   id_cliente  804293 non-null  object
 2   id_issue    804302 non-null  int64 
 3   canal       804302 non-null  object
 4   tag         804302 non-null  object
dtypes: int64(1), object(4)
memory usage: 30.7+ MB


In [4]:
#import tabela 2 - dados para de-para
df2 = pd.read_csv('/content/drive/My Drive/case_will/de_para_tag_hierarquia.csv')
df2.head()

,tag,ds_hierarquia_a,ds_hierarquia_b
0,w3 - solicitacao - 2 via cartao fraudado,Cartão,Segunda via cartão
1,wb-cdu,Cartão,Lançamentos duplicados
2,w3 - solicitacao - cancelar insatisfacao,Cartão,Cancelamento de conta
3,w3 - duvida - cancelamento de compra,Cartão,Cancelamento de compra no crédito
4,wb-cf,Cartão,Compensação fatura


In [5]:
#infos tabela 2
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tag              481 non-null    object
 1   ds_hierarquia_a  481 non-null    object
 2   ds_hierarquia_b  481 non-null    object
dtypes: object(3)
memory usage: 11.4+ KB


In [6]:
#check de tags para canal email (sem regra para tag única)
df1[df1['canal'] == 'email']

,dt_data,id_cliente,id_issue,canal,tag
273333,2024-10-17 20:18:35.000,671cc644-0ff5-4a88-aff2-18d918d45ad8-765gh,8462101,email,w3 - problema - documento proposta
273334,2024-10-17 20:18:35.000,671cc644-0ff5-4a88-aff2-18d918d45ad8-765gh,8462101,email,w3 - problema - documento proposta
273335,2024-10-17 20:18:35.000,671cc644-0ff5-4a88-aff2-18d918d45ad8-765gh,8462101,email,w3 - duvida - cadastro web
286770,2024-10-21 21:04:46.000,0f59978c-f999-4767-8586-ef0f8e88229c-765gh,8607813,email,w3 - duvida - resultado da proposta
286771,2024-10-21 21:04:46.000,0f59978c-f999-4767-8586-ef0f8e88229c-765gh,8607813,email,w3 - duvida - cadastro web
732404,2024-11-01 18:01:10.000,2c0ed7ad-6d81-466d-a1a6-18f29a654559-765gh,8929572,email,w3 - duvida - cadastro web


In [7]:
#criando dataframe principal com merge dos iniciais
df = pd.merge(df1, df2, how='inner', left_on='tag', right_on='tag')
df.head()

,dt_data,id_cliente,id_issue,canal,tag,ds_hierarquia_a,ds_hierarquia_b
0,2024-10-07 17:32:50.000,22df4146-1b0b-42e1-bc64-1905e921a56a-765gh,8095572,chat,w3 - reclamacao - resultado analise de contest...,Fraude,Resultado análise de fraude
1,2024-12-03 19:09:43.000,019226ac-f859-7590-9cfe-56d56f5e8f59-765gh,9872055,chat,w3 - reclamacao - resultado analise de contest...,Fraude,Resultado análise de fraude
2,2024-10-16 18:26:47.000,c73c1daa-efd4-4a07-9671-1e52fb008601-765gh,8428710,chat,w3 - duvida - pix no credito,Cartão,Todos os itens de pix no crédito
3,2024-12-12 15:34:49.000,01923e84-7191-7bf0-bd29-0aeb1417f4e1-765gh,10153733,chat,w3 - problema - desacordo comercial,Cartão,Problema com compra
4,2024-12-05 12:37:52.000,881b24fc-23d1-4dd3-80ab-00f636cecdfa-765gh,9934026,chat,wb-caf,Cartão,Compensação fatura


In [8]:
#infos dataframe mergeado - check de nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804302 entries, 0 to 804301
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   dt_data          804302 non-null  object
 1   id_cliente       804293 non-null  object
 2   id_issue         804302 non-null  int64 
 3   canal            804302 non-null  object
 4   tag              804302 non-null  object
 5   ds_hierarquia_a  804302 non-null  object
 6   ds_hierarquia_b  804302 non-null  object
dtypes: int64(1), object(6)
memory usage: 43.0+ MB


In [9]:
#Checando período de análise:
df['dt_data'].min()

'2024-10-01 00:00:12.000'

In [10]:
df['dt_data'].max()

'2024-12-31 23:53:19.000'

In [11]:
# Conversão dt_data em datetime
df['dt_data'] = pd.to_datetime(df['dt_data'])

#Colunas de particionamento
df['dia'] = df['dt_data'].dt.date
df['semana'] = df['dt_data'].dt.to_period('W').apply(lambda r: r.start_time.date())
df['mes'] = df['dt_data'].dt.to_period('M').apply(lambda r: r.start_time.date())
df['hora'] = df['dt_data'].dt.hour

df.head()

,dt_data,id_cliente,id_issue,canal,tag,ds_hierarquia_a,ds_hierarquia_b,dia,semana,mes,hora
0,2024-10-07 17:32:50,22df4146-1b0b-42e1-bc64-1905e921a56a-765gh,8095572,chat,w3 - reclamacao - resultado analise de contest...,Fraude,Resultado análise de fraude,2024-10-07,2024-10-07,2024-10-01,17
1,2024-12-03 19:09:43,019226ac-f859-7590-9cfe-56d56f5e8f59-765gh,9872055,chat,w3 - reclamacao - resultado analise de contest...,Fraude,Resultado análise de fraude,2024-12-03,2024-12-02,2024-12-01,19
2,2024-10-16 18:26:47,c73c1daa-efd4-4a07-9671-1e52fb008601-765gh,8428710,chat,w3 - duvida - pix no credito,Cartão,Todos os itens de pix no crédito,2024-10-16,2024-10-14,2024-10-01,18
3,2024-12-12 15:34:49,01923e84-7191-7bf0-bd29-0aeb1417f4e1-765gh,10153733,chat,w3 - problema - desacordo comercial,Cartão,Problema com compra,2024-12-12,2024-12-09,2024-12-01,15
4,2024-12-05 12:37:52,881b24fc-23d1-4dd3-80ab-00f636cecdfa-765gh,9934026,chat,wb-caf,Cartão,Compensação fatura,2024-12-05,2024-12-02,2024-12-01,12


In [12]:
#Entendendo distribuição geral por canal de abertura

df_canal = df['canal'].value_counts(normalize=True).mul(100).reset_index()
df_canal.columns = ['canal', 'percentual_chamados']

fig = px.bar(df_canal, x='canal', y='percentual_chamados',  text='percentual_chamados', title='Distribuição percentual por canal',  labels={'percentual': '%'},
          color_discrete_sequence=["yellow", "gray"])

fig.update_traces(texttemplate='%{text:.4f}%', textposition='outside')
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [13]:
#Entendendo distribuição geral por hierarquia (A)

df_hierarquia_a = df['ds_hierarquia_a'].value_counts(normalize=True).mul(100).reset_index()
df_hierarquia_a.columns = ['ds_hierarquia_a', 'percentual_chamados']

fig = px.bar(df_hierarquia_a, x='ds_hierarquia_a', y='percentual_chamados',  text='percentual_chamados', title='Distribuição percentual por hierarquia (A)',  labels={'percentual': '%'},
          color='ds_hierarquia_a', color_discrete_sequence=["yellow", "orange", "gray", "black"])

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [14]:
df['ds_hierarquia_b'].value_counts()

,count
ds_hierarquia_b,
Cancelamento de conta,78871
Não reconhecimento de compra no crédito,66988
Cancelamento de compra no crédito,49846
Segunda via cartão,45545
Compensação fatura,42968
...,...
Cupom loja will,9
Desinteresse comercial,7
Pix no crédito,6


In [15]:
#Entendendo distribuição geral por hierarquia (B) - top10

df_hierarquia_b = df['ds_hierarquia_b'].value_counts(normalize=True).mul(100).nlargest(10).reset_index()
df_hierarquia_b.columns = ['ds_hierarquia_b', 'percentual_chamados']

fig = px.bar(df_hierarquia_b, x='ds_hierarquia_b', y='percentual_chamados',  text='percentual_chamados', title='Top 10 Chamado por Hierarquia (B)', labels={'percentual': '%'},
          color='ds_hierarquia_b', color_discrete_sequence=["#000000", "#1C1C1C", "#4F4F4F", "#7F7F7F", "#BEBEBE","#FFD700", "#FFC300", "#FFEA00", "#FFF176", "#FFF9C4"])

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_layout(xaxis_tickangle=-90)
fig.update_layout(yaxis=dict(range=[0, 12]))

fig.show()

In [16]:
df['tag'].value_counts()

,count
tag,
w3 - duvida - estorno,34452
w3 - problema - nao reconhece compra online,29273
w3 - duvida - detalhamento de faturas,22098
w3 - solicitacao - cancelar limite baixo,18214
w3 - duvida - nao elegivel pix credito,17543
...,...
w3 - reclamacao - bloqueio alteracao cadastral,1
w3 - problema - proposta web,1
w3 - problema - contratacao de seguro debito em conta,1


In [17]:
#Entendendo distribuição geral por tags - top10

df_tag = df['tag'].value_counts(normalize=True).mul(100).nlargest(10).reset_index()
df_tag.columns = ['tag', 'percentual_chamados']

fig = px.bar(df_tag, x='tag', y='percentual_chamados',  text='percentual_chamados', title='Top 10 Chamado por tag', labels={'percentual': '%'},
          color='tag', color_discrete_sequence=["#000000", "#1C1C1C", "#4F4F4F", "#7F7F7F", "#BEBEBE","#FFD700", "#FFC300", "#FFEA00", "#FFF176", "#FFF9C4"])

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_layout(xaxis_tickangle=-90)
fig.update_layout(yaxis=dict(range=[0, 12]))

fig.show()

In [18]:
#Entendendo hierarquias associadas as principais tags

df_tag_hier = pd.merge(df_tag, df2, how='inner', left_on='tag', right_on='tag')
df_tag_hier.head(10)

,tag,percentual_chamados,ds_hierarquia_a,ds_hierarquia_b
0,w3 - duvida - estorno,4.283466,Cartão,Cancelamento de compra no crédito
1,w3 - problema - nao reconhece compra online,3.639553,Fraude,Não reconhecimento de compra no crédito
2,w3 - duvida - detalhamento de faturas,2.747475,Cartão,Faturas futuras
3,w3 - solicitacao - cancelar limite baixo,2.264572,Cartão,Cancelamento de conta
4,w3 - duvida - nao elegivel pix credito,2.181146,Cartão,Todos os itens de pix no crédito
5,w3 - solicitacao - cancelar motivos pessoais,2.115996,Cartão,Cancelamento de conta
6,w3 - reclamacao - desconhece recebimento,1.959463,Cartão,Problema no recebimento do cartão
7,w3 - duvida - antecipacao fatura,1.794102,Cartão,Antecipação de Fatura
8,w3 - solicitacao - resultado analise de contes...,1.751208,Fraude,Resultado análise de fraude
9,w3 - duvida - valor total da fatura,1.675117,Cartão,Não entendimento fatura


In [19]:
#Entendendo volume de chamados por hora do dia

df_hora = df.groupby('hora').agg(
    chamados=('id_issue', 'count'),
    clientes=('id_cliente', 'nunique')
).reset_index()

fig = px.line(df_hora, x='hora', y=['chamados','clientes'], title='Chamados e Clientes por Hora do Dia',
              color_discrete_sequence=['yellow', 'black'])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [20]:
#Entendendo volume de chamados diário

df_dia = df.groupby(['dia', 'ds_hierarquia_a']).agg(
    chamados=('id_issue', 'count'),
    clientes=('id_cliente', 'nunique')
).reset_index()

fig = px.line(df_dia, x="dia", y="chamados", color="ds_hierarquia_a", title='Volume de chamados diário por hierarquia (A)',
              color_discrete_sequence=["yellow", "orange", "gray", "black"])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [21]:
#Entendendo volume de clientes diário

fig = px.line(df_dia, x="dia", y="clientes", color="ds_hierarquia_a", title='Volume de clientes diário por hierarquia (A)',
              color_discrete_sequence=["yellow", "orange", "gray", "black"])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [22]:
#Percebida sazonalidade semanal, entendendo volume de chamados por semana

df_semana = df.groupby(['semana', 'ds_hierarquia_a']).agg(
    chamados=('id_issue', 'count'),
    clientes=('id_cliente', 'nunique')
).reset_index()

df_semana['chamados_por_cliente'] = (df_semana['chamados'] / df_dia['clientes']).round(2)


fig = px.line(df_semana, x="semana", y="chamados", color="ds_hierarquia_a", title='Volume de chamados semanal por hierarquia (A)',
              color_discrete_sequence=["yellow", "orange", "gray", "black"])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [23]:
#Criando acompanhamento WoW

df_wow = (df.groupby(['semana', 'ds_hierarquia_a'])
      .agg(chamados=('id_issue', 'count'))
      .reset_index())

df_wow = df_wow.sort_values(['ds_hierarquia_a', 'semana'])
df_wow['chamados_w-1'] = df_wow.groupby('ds_hierarquia_a')['chamados'].shift(1)

df_wow['variacao_percentual'] = ((df_wow['chamados'] - df_wow['chamados_w-1']) /
                               df_wow['chamados_w-1']) * 100

df_wow['variacao_percentual'] = df_wow['variacao_percentual'].fillna(0)
df_wow['variacao_percentual'] = df_wow['variacao_percentual'].round(1)

def color_variacao(val):
    if val > 0:
        color = 'red'     # aumento
    elif val < 0:
        color = 'green'   # redução
    else:
        color = ''        # neutro
    return f'color: {color}'

#df_wow.style.applymap(color_variacao, subset=['variacao_percentual'])

df_wow_color = df_wow.style\
    .applymap(color_variacao, subset=['variacao_percentual'])\
    .format({'variacao_percentual': '{:.1f}', 'chamados_w-1': '{:.1f}'})

df_wow_color

,semana,ds_hierarquia_a,chamados,chamados_w-1,variacao_percentual
0,2024-09-30,Cartão,35488,nan,0.0
4,2024-10-07,Cartão,51419,35488.0,44.9
8,2024-10-14,Cartão,48238,51419.0,-6.2
12,2024-10-21,Cartão,45777,48238.0,-5.1
16,2024-10-28,Cartão,39021,45777.0,-14.8
20,2024-11-04,Cartão,50997,39021.0,30.7
24,2024-11-11,Cartão,43818,50997.0,-14.1
28,2024-11-18,Cartão,47272,43818.0,7.9
32,2024-11-25,Cartão,44628,47272.0,-5.6
36,2024-12-02,Cartão,49680,44628.0,11.3


In [24]:
#Percebida sazonalidade semanal, entendendo volume de clientes por semana

fig = px.line(df_semana, x="semana", y="clientes", color="ds_hierarquia_a", title='Volume de clientes semanal por hierarquia (A)',
              color_discrete_sequence=["yellow", "orange", "gray", "black"])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

In [25]:
#Entendendo ratio chamados/cliente semanalmente

fig = px.line(df_semana, x="semana", y="chamados_por_cliente", color="ds_hierarquia_a", title='Ratio chamados/clientes semanal por hierarquia (A)',
              color_discrete_sequence=["yellow", "orange", "gray", "black"])
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.show()

# Parte 2

Baseado no diagnóstico realizado na Parte 1, crie um indicador de ‘’recontato’’ e sugira iniciativas para reduzi-lo.

Além disso, defina targets (via benchmark) e resuma seus insights em um diagnóstico da situação atual.


As iniciativas levantadas precisam ser priorizadas, com uma metodologia à escolha do candidato, e devem contemplar potenciais impactos nas demais verticais de negócio.


Entregáveis esperados: código sql com a o indicador de recontato + indicador de recontato + target + metodologia de priorização + iniciativas priorizadas + verticais impactadas com planos de trabalho

In [26]:
#Entendendo percentual de recontato para 2, 3 e > 3, independete da hierarquia e considerando 3 meses

query = """
with chamados_ordem as (
    select
        dt_data,
        id_issue,
        id_cliente,
        row_number() over (partition by id_cliente order by dt_data) as num_chamado_cliente
    from df
)
select
    count(*) as total_chamados,
    round(100.0 * sum(case when num_chamado_cliente > 1 then 1 else 0 end) / count(*), 2) as percentual_recontato,
    round(100.0 * sum(case when num_chamado_cliente > 2 then 1 else 0 end) / count(*), 2) as percentual_2_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 3 then 1 else 0 end) / count(*), 2) as percentual_3_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 4 then 1 else 0 end) / count(*), 2) as percentual_4_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 5 then 1 else 0 end) / count(*), 2) as percentual_5_recontatos
from chamados_ordem;
"""

resultado1 = duckdb.query(query).to_df()

resultado1

,total_chamados,percentual_recontato,percentual_2_recontatos,percentual_3_recontatos,percentual_4_recontatos,percentual_5_recontatos
0,804302,50.1,26.56,15.33,9.41,6.06


In [27]:
#Entendendo percentual de recontato para 2, 3 e > 3, independete da hierarquia e por mês

query = """
with chamados_ordem as (
    select
        dt_data,
        mes,
        id_issue,
        id_cliente,
        row_number() over (partition by id_cliente order by dt_data) as num_chamado_cliente
    from df
)
select
    mes,
    count(*) as total_chamados,
    round(100.0 * sum(case when num_chamado_cliente > 1 then 1 else 0 end) / count(*), 2) as percentual_recontato,
    round(100.0 * sum(case when num_chamado_cliente > 2 then 1 else 0 end) / count(*), 2) as percentual_2_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 3 then 1 else 0 end) / count(*), 2) as percentual_3_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 4 then 1 else 0 end) / count(*), 2) as percentual_4_recontatos,
    round(100.0 * sum(case when num_chamado_cliente > 5 then 1 else 0 end) / count(*), 2) as percentual_5_recontatos
from chamados_ordem
group by mes
order by mes
"""

resultado2 = duckdb.query(query).to_df()

resultado2

,mes,total_chamados,percentual_recontato,percentual_2_recontatos,percentual_3_recontatos,percentual_4_recontatos,percentual_5_recontatos
0,2024-10-01,284626,45.55,21.64,11.21,6.14,3.55
1,2024-11-01,265318,52.23,28.55,16.79,10.42,6.74
2,2024-12-01,254358,52.96,29.98,18.43,12.03,8.16


In [28]:
#Entendendo % de clientes que faz recontato

query = """
with chamados_por_cliente as (
    select
        id_cliente,
        count(distinct id_issue) as qtd_chamados
    from df
    group by id_cliente
),

clientes_recontato as (
    select
        qtd_chamados,
        count(*) as num_clientes
    from chamados_por_cliente
    group by qtd_chamados
),

clientes_total as (
    select count (*) as total_clientes from chamados_por_cliente
)

select
    t.qtd_chamados,
    t.num_clientes,
    round( (t.num_clientes * 100.0) / ct.total_clientes, 2) as percentual_clientes_recontato
from clientes_recontato t
cross join clientes_total ct
where t.qtd_chamados BETWEEN 1 AND 10
order by t.qtd_chamados;
"""

resultado3 = duckdb.query(query).to_df()

resultado3

,qtd_chamados,num_clientes,percentual_clientes_recontato
0,1,291315,72.58
1,2,68765,17.13
2,3,22684,5.65
3,4,9124,2.27
4,5,4302,1.07
5,6,2189,0.55
6,7,1138,0.28
7,8,673,0.17
8,9,408,0.10
9,10,254,0.06


In [29]:
#Entendendo percentual de recontato em uma mesma hierarquia (B) por mês

query = """
with chamados_ordem as (
    select
        id_issue,
        id_cliente,
        ds_hierarquia_b,
        dt_data,
        mes,
        row_number() over (
            partition by id_cliente, ds_hierarquia_b
            order by dt_data
        ) as num_chamado_cliente_hierarquia
    from df
)
select
    mes,
    count(*) as total_chamados,
    sum(case when num_chamado_cliente_hierarquia > 1 then 1 else 0 end) as chamados_recontato_mesma_hierarquia,
    round(100.0 * sum(case when num_chamado_cliente_hierarquia > 1 then 1 else 0 end) / count(*), 2) as percentual_recontato_mesma_hierarquia_b
from chamados_ordem
group by mes
"""

resultado4 = duckdb.query(query).to_df()

resultado4

,mes,total_chamados,chamados_recontato_mesma_hierarquia,percentual_recontato_mesma_hierarquia_b
0,2024-10-01,284626,70306.0,24.70
1,2024-11-01,265318,73792.0,27.81
2,2024-12-01,254358,69712.0,27.41


In [30]:
#Entendendo hierarquias (B) com maior volume de chamados de recontato na mesma hierarquia (B)

query = """
with chamados_ordem as (
    select
        id_issue,
        id_cliente,
        ds_hierarquia_a,
        ds_hierarquia_b,
        dt_data,
        row_number() over (
            partition by id_cliente, ds_hierarquia_b
            order by dt_data
        ) as num_chamado_cliente_hierarquia_b
    from df
)
select
    ds_hierarquia_a,
    ds_hierarquia_b,
    count(*) as volume_chamados,
    sum(case when num_chamado_cliente_hierarquia_b > 1 then 1 else 0 end) as chamados_recontato,
    round(100.0 * sum(case when num_chamado_cliente_hierarquia_b > 1 then 1 else 0 end) / count(*), 2) as percentual_recontato
from chamados_ordem
group by ds_hierarquia_b, ds_hierarquia_a
order by chamados_recontato desc
limit 10
"""

resultado5 = duckdb.query(query).to_df()

resultado5

,ds_hierarquia_a,ds_hierarquia_b,volume_chamados,chamados_recontato,percentual_recontato
0,Cartão,Cancelamento de conta,78871,30114.0,38.18
1,Fraude,Não reconhecimento de compra no crédito,59120,22725.0,38.44
2,Cartão,Cancelamento de compra no crédito,49846,17117.0,34.34
3,Cartão,Segunda via cartão,45545,12380.0,27.18
4,Cartão,Informações sobre fatura,40495,10016.0,24.73
5,Cartão,Problema no recebimento do cartão,24119,9394.0,38.95
6,Cartão,Compensação fatura,42968,8034.0,18.70
7,Fraude,Resultado análise de fraude,17409,7316.0,42.02
8,Cartão,Desbloqueio de cartão,26725,7044.0,26.36
9,Cartão,Todos os itens de pix no crédito,26220,6859.0,26.16


# Parte 3

Crie um modelo de série temporal para prever o volume de atendimentos pelos próximos 3 meses.

Entregáveis esperados: modelo em código (Python)

In [31]:
import plotly.graph_objects as go
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
from prophet.plot import plot_plotly, plot_components_plotly

In [32]:
#Criando dataset de série temporal

df_pred = df.groupby(['dia']).agg(
    quantidade_chamados=('id_issue', 'count')
).reset_index()

df_pred = df_pred.rename(columns={
    'dia': 'ds',
    'quantidade_chamados': 'y'
})

In [33]:
# Criando DataFrame com feriados do Brasil
feriados_padrao = make_holidays_df(year_list=[2024, 2025], country='BR')

feriados_custom = pd.DataFrame({
    'holiday': ['Véspera_Natal', 'Véspera_AnoNovo', 'Carnaval', 'Carnaval', 'Carnaval', 'Carnaval'],
    'ds': pd.to_datetime(['2024-12-24', '2024-12-31', '2025-03-01', '2025-03-02', '2025-03-03', '2025-03-04']),
    'lower_window': -1,
    'upper_window': 0})

feriados_completos = pd.concat([feriados_padrao, feriados_custom], ignore_index=True)

In [34]:
#instanciando modelo

m = Prophet(
    seasonality_mode="additive", holidays=feriados_completos)

m.fit(df_pred)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu1q0rir_/b7rnnfvq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu1q0rir_/7u1avrzj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82770', 'data', 'file=/tmp/tmpu1q0rir_/b7rnnfvq.json', 'init=/tmp/tmpu1q0rir_/7u1avrzj.json', 'output', 'file=/tmp/tmpu1q0rir_/prophet_modelpajjv_z5/prophet_model-20250811114406.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:44:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:44:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [35]:
#definindo período futuro a ser predito

future = m.make_future_dataframe(periods = 90)
future.tail()

,ds
177,2025-03-27
178,2025-03-28
179,2025-03-29
180,2025-03-30
181,2025-03-31


In [36]:
#Executando prefições com faixa de erro

forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
177,2025-03-27,9407.134763,7808.038861,10953.504888
178,2025-03-28,8865.611946,7479.747592,10557.718410
179,2025-03-29,6126.681111,4430.429741,7705.768679
180,2025-03-30,3922.469836,2310.470642,5456.607877
181,2025-03-31,10249.860590,8673.934219,11943.606804


In [37]:
#Plotando valores reais e predições

fig = plot_plotly(m, forecast)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.show()

In [38]:
#Plotando componentes da série temporal

fig = plot_components_plotly(m, forecast)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.show()

In [39]:
#Consolidando mensalmente
forecast['mes'] = forecast['ds'].dt.to_period('M').apply(lambda r: r.start_time)

df_fc_mensal = forecast.groupby(['mes']).agg(
    fc_chamados=('yhat', 'sum')
).reset_index().round(0)

df_fc_mensal

,mes,fc_chamados
0,2024-10-01,288910.0
1,2024-11-01,263528.0
2,2024-12-01,251840.0
3,2025-01-01,272929.0
4,2025-02-01,239428.0
5,2025-03-01,255506.0


In [40]:
fig = px.line(df_fc_mensal, x="mes", y="fc_chamados", title='Predição de volume de chamados', text='fc_chamados',
              color_discrete_sequence=["yellow"])

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_traces(line=dict(width=4))

fig.show()